Examen PySpark
Instrucciones: Lea cuidadosamente las preguntas, escriba el código correspondiente y ejecútelo para mostrar sus resultados.

Importante: Todos los ejercicios deberán realizarse con funciones de NumPy, Pandas o PySpark (no podrán crearse vistas temporales para realizarse en SQL, salvo que se indique lo contrario).

1.1 Utilizando NumPy, construya un arreglo con 50 elementos aleatorios distribuidos de forma normal con media 50 y desviación estándar 10. Imprima el arreglo.

In [25]:
import findspark
import pyspark
import numpy as np
import pandas as pd

In [2]:
a = np.random.normal(50, 10, 50)
print(a)

[50.04316237 43.72370767 62.83363972 59.23240647 54.45381436 56.08564635
 56.92165967 40.85530363 47.01727796 52.92960562 56.67032375 41.94263761
 63.18626006 35.82079251 56.97839199 50.53012189 54.64222832 39.93023964
 48.95581834 64.80919255 49.30900384 44.1557694  34.94099168 62.27771462
 61.8872053  52.00162512 61.12248833 37.28464461 60.57197059 57.96061479
 77.56091868 62.89064105 48.64702223 44.09035403 40.14135355 58.34462852
 56.68706383 60.60498466 52.31258585 59.62115622 69.83035174 49.37941611
 51.23289531 40.80846773 51.80886103 66.88520535 46.26600815 62.25856068
 37.39785911 61.05474058]


1.2. Construya el objeto de Spark (Core) que le permita trabajar con objetos RDD.

In [3]:
findspark.init()
sc = pyspark.SparkContext.getOrCreate()


1.3. Convierta el arreglo de NumPy a un RDD con 2 particiones. Muestre los primeros 5 elementos.

In [4]:
myrdd = sc.parallelize(a, 2)
myrdd.take(5)

[50.04316237493861,
 43.723707667524536,
 62.83363972365511,
 59.232406469864344,
 54.45381435656503]

1.4. Suponiendo que los datos de la lista miden grados Fahrenheit, aplique una función lambda al RDD que convierta las mediciones a grados Centígrados. Muestre los primeros 5 elementos.

In [7]:
centigrados = myrdd.map(lambda F: ((F - 32) * 5 / 9))
centigrados.take(5)

[10.023979097188118,
 6.51317092640252,
 17.12979984647506,
 15.129114705480191,
 12.474341309202796]

1.5. Utilice una función Lambda para mostrar únicamente las temperaturas mayores a 15 grados Centigrados.

In [14]:
centigrados.filter(lambda x: x > 15).collect()

[17.12979984647506,
 15.129114705480191,
 17.325700030770577,
 18.227329195564547,
 16.820952565362514,
 16.604002946445892,
 16.17916018212483,
 15.873316997159456,
 25.31162149082154,
 17.161467249737736,
 15.891658144951244,
 15.345086787347526,
 21.01686207868806,
 19.38066963880271,
 16.810311486728235,
 16.141522543877553]

1.6. Calcule la temperatura media en grados Centígrados.

In [18]:
centigrados.mean()

11.743303702068593

1.7. Obtenga las 3 temperaturas más altas en grados Centígrados.

In [23]:
centigrados.sortBy(lambda x:x, ascending=False).take(3)

[25.31162149082154, 21.01686207868806, 19.38066963880271]

Bloque 2: Spark SQL

2.1. Utilizando Numpy, construya un arreglo con 50 números enteros entre 1 y 3 (1 y 3 incluidos).

In [34]:
b = np.random.randint(1, 4, 50)
print(b)

[1 2 3 1 2 2 1 2 3 2 3 2 3 2 1 3 2 1 2 2 2 1 3 1 1 3 1 1 2 2 2 3 1 2 1 1 2
 3 1 2 2 2 3 2 2 3 3 2 1 2]


array([1, 2, 3, 1, 2, 2, 1, 2, 3, 2, 3, 2, 3, 2, 1, 3, 2, 1, 2, 2, 2, 1,
       3, 1, 1, 3, 1, 1, 2, 2, 2, 3, 1, 2, 1, 1, 2, 3, 1, 2, 2, 2, 3, 2,
       2, 3, 3, 2, 1, 2])

2.2. Construya un dataframe en Pandas utilizando los arreglos de 2.1 y 1.1. Asigne los nombres "dia" y "temp". Muestre los primeros 5 elementos.

In [38]:
numpy_data = np.array([b, a])
df = pd.DataFrame(numpy_data.T, columns=["dia", "temp"])
df.dia = df.dia.astype(int)
df.head(5)

,dia,temp
0,1,50.043162
1,2,43.723708
2,3,62.833640
3,1,59.232406
4,2,54.453814


2.3. Construya el objeto de Spark (SQL) que le permita trabajar con los dataframes de Spark.

In [40]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
    master("local[2]").\
    appName("challenge").\
    config('spark.default.parallelism',2).\
    getOrCreate()

2.4. Convierta el dataframe de Pandas a un dataframe de Spark, definiendo explícitamente el esquema/estructura (utilice el tipo entero para el día y el tipo doble para la temperatura). Muestre los primeros 5 registros.

In [49]:
from pyspark.sql.types import IntegerType, StructType, StructField, DoubleType
 
miesquema = StructType([
     StructField("dia",IntegerType(),True),
     StructField("temp",DoubleType(),True)])

df_spark = spark.createDataFrame(df, miesquema)
df_spark.show(5)

+---+------------------+
|dia|              temp|
+---+------------------+
|  1| 50.04316237493861|
|  2|43.723707667524536|
|  3| 62.83363972365511|
|  1|59.232406469864344|
|  2| 54.45381435656503|
+---+------------------+
only showing top 5 rows



2.5. Partiendo del dataframe en Spark, construya un dataframe con el promedio de temperatura agrupado por día. El dataframe deberá contener únicamente las columnas "dia" y "temp_prom" (con esos nombres). Muestre la tabla resultante.

In [60]:
gb = df_spark.groupBy("dia").agg({'temp': 'mean'}).toDF("dia", "temp_prom")
gb.show(3)

+---+-----------------+
|dia|        temp_prom|
+---+-----------------+
|  1|51.11180877756445|
|  3|54.58833122497515|
|  2|53.70261855752193|
+---+-----------------+



2.6. Repita el ejercicio anterior registrando una vista temporal y ejecutando el código SQL correspondiente. Muestre la tabla resultante.

In [59]:
df_spark.createOrReplaceTempView("temperaturas")
spark.sql("select dia, avg(temp) as temp_prom FROM temperaturas GROUP BY dia").show(3)

+---+-----------------+
|dia|        temp_prom|
+---+-----------------+
|  1|51.11180877756445|
|  3|54.58833122497515|
|  2|53.70261855752193|
+---+-----------------+



2.7. Combine los valores del dataframe anterior con el original. El dataframe resultante no deberá contener columnas repetidas y tendrá que estar ordenado de forma ascendente por día y temperatura. Muestre los primeros 5 elementos.

In [62]:
joined = df_spark.join(gb, "dia").orderBy("dia", "temp")
joined.show(5)

+---+------------------+-----------------+
|dia|              temp|        temp_prom|
+---+------------------+-----------------+
|  1| 37.28464461164833|51.11180877756445|
|  1|37.397859107548456|51.11180877756445|
|  1|39.930239643141576|51.11180877756445|
|  1| 40.14135355104299|51.11180877756445|
|  1|44.155769403252734|51.11180877756445|
+---+------------------+-----------------+
only showing top 5 rows



2.8. Añada una columna adicicional con la diferencia entre la temperatura y su media. Asigne el nombre "resid". Muestre los primeros 5 elementos.

In [67]:
residuos = joined.withColumn("resid", joined.temp- joined.temp_prom)
residuos.show(5)

+---+------------------+-----------------+-------------------+
|dia|              temp|        temp_prom|              resid|
+---+------------------+-----------------+-------------------+
|  1| 37.28464461164833|51.11180877756445|-13.827164165916116|
|  1|37.397859107548456|51.11180877756445|-13.713949670015992|
|  1|39.930239643141576|51.11180877756445|-11.181569134422872|
|  1| 40.14135355104299|51.11180877756445|-10.970455226521459|
|  1|44.155769403252734|51.11180877756445| -6.956039374311715|
+---+------------------+-----------------+-------------------+
only showing top 5 rows



2.9. Construya un dataframe con todos los registros que posean residuales negativos. Muestre los primeros 5 elementos.

In [71]:
residuos_filtered = residuos.filter(residuos.resid < 0)
residuos_filtered.show(5)

+---+------------------+-----------------+-------------------+
|dia|              temp|        temp_prom|              resid|
+---+------------------+-----------------+-------------------+
|  1| 37.28464461164833|51.11180877756445|-13.827164165916116|
|  1|37.397859107548456|51.11180877756445|-13.713949670015992|
|  1|39.930239643141576|51.11180877756445|-11.181569134422872|
|  1| 40.14135355104299|51.11180877756445|-10.970455226521459|
|  1|44.155769403252734|51.11180877756445| -6.956039374311715|
+---+------------------+-----------------+-------------------+
only showing top 5 rows



2.10. Guarde el dataframe resultante en formato JSON. En caso de que el archivo ya exista, deberá sobreescribirse.

In [73]:
residuos_filtered.write.mode( "overwrite" ).json( "solution" )

Bloque 3: Spark MLlib
En esta sección se evalúan los conocimientos de Spark MLlib. Si bien son necesarios los conociemientos en Machine Learning, el candidato no será evaluado por la calidad del modelo producido.
3.1. Cargue los datos del archivo 'data.csv'.

3.2. Realicé un análisis exploratorio preliminar de los datos (estadísticos básicos de las columnas).

3.3. Obtenga el conjunto de datos con el vector de variables independientes y la variable dependiente (churn). Por simplicidad, es suficiente que seleccione únicamente las variables numéricas. Muestre los primeros 5 elementos.

3.4. Realicé la separación en los conjuntos de entrenamiento y prueba con una proporción 70-30. Muestre los primeros 5 elementos de cada conjunto de datos.

3.5. Ajuste un modelo de regresión logística con los hiperparámetros por defecto. Muestre los estadísticos descriptivos de las predicciones contenidas en el resumen del modelo.

3.6. Evalúe los resultados en el conjunto de prueba. Muestre las primeras 5 predicciones.

3.7. Para evaluar el desempeño del modelo, obtenga el valor del indicador auROC (área debajo de la curva ROC).

3.8. Cargue los datos del archivo 'data_new.csv' y obtenga las predicciones sobre ese conjunto de datos utilizando los objetos construidos previamente. Muestre los primeros 5 elementos.

In [77]:
training = spark.read.csv('data.csv',header=True,inferSchema=True, sep=',')
training.describe().show()


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [78]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Total_Purchase", "Account_Manager","Years","Num_Sites"],outputCol="Independent Features")
output=featureassembler.transform(training)
output.take(5)

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|Independent Features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|[42.0,11066.8,0.0...|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|[41.0,11916.22,0....|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|[38.0,12884.75,0....|
|      Phillip White|4

In [79]:
finalized_data=output.select("Independent Features","Churn")
finalized_data.take(5)

[Row(Independent Features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0]), Churn=1),
 Row(Independent Features=DenseVector([41.0, 11916.22, 0.0, 6.5, 11.0]), Churn=1),
 Row(Independent Features=DenseVector([38.0, 12884.75, 0.0, 6.67, 12.0]), Churn=1),
 Row(Independent Features=DenseVector([42.0, 8010.76, 0.0, 6.71, 10.0]), Churn=1),
 Row(Independent Features=DenseVector([37.0, 9191.58, 0.0, 5.56, 9.0]), Churn=1)]

In [80]:
from pyspark.ml.classification import LogisticRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.7,0.3])
train_data.take(5)


[Row(Independent Features=DenseVector([22.0, 11254.38, 1.0, 4.96, 8.0]), Churn=0),
 Row(Independent Features=DenseVector([27.0, 8628.8, 1.0, 5.3, 7.0]), Churn=0),
 Row(Independent Features=DenseVector([28.0, 8670.98, 0.0, 3.99, 6.0]), Churn=0),
 Row(Independent Features=DenseVector([28.0, 9090.43, 1.0, 5.74, 10.0]), Churn=0),
 Row(Independent Features=DenseVector([28.0, 11204.23, 0.0, 3.67, 11.0]), Churn=0)]

In [82]:
regressor=LogisticRegression(featuresCol='Independent Features', labelCol='Churn')
regressor=regressor.fit(train_data)

In [83]:
### Coefficients
regressor.coefficients

DenseVector([0.0696, 0.0001, 0.7817, 0.5049, 1.2076])

In [84]:
### Intercepts
regressor.intercept

-19.660683811712982

In [89]:
### Prediction
pred_results=regressor.evaluate(test_data)
pred_results.predictions.take(5)

[Row(Independent Features=DenseVector([25.0, 9672.03, 0.0, 5.49, 8.0]), Churn=0, rawPrediction=DenseVector([5.0034, -5.0034]), probability=DenseVector([0.9933, 0.0067]), prediction=0.0),
 Row(Independent Features=DenseVector([26.0, 8787.39, 1.0, 5.42, 11.0]), Churn=1, rawPrediction=DenseVector([0.609, -0.609]), probability=DenseVector([0.6477, 0.3523]), prediction=0.0),
 Row(Independent Features=DenseVector([26.0, 8939.61, 0.0, 4.54, 7.0]), Churn=0, rawPrediction=DenseVector([6.6579, -6.6579]), probability=DenseVector([0.9987, 0.0013]), prediction=0.0),
 Row(Independent Features=DenseVector([28.0, 11128.95, 1.0, 5.12, 8.0]), Churn=0, rawPrediction=DenseVector([4.1268, -4.1268]), probability=DenseVector([0.9841, 0.0159]), prediction=0.0),
 Row(Independent Features=DenseVector([29.0, 9378.24, 0.0, 4.93, 8.0]), Churn=0, rawPrediction=DenseVector([5.0226, -5.0226]), probability=DenseVector([0.9935, 0.0065]), prediction=0.0)]

In [88]:
 regressor.summary.roc.show()

+--------------------+--------------------+
|                 FPR|                 TPR|
+--------------------+--------------------+
|                 0.0|                 0.0|
|                 0.0| 0.00909090909090909|
|                 0.0| 0.01818181818181818|
|                 0.0| 0.02727272727272727|
|                 0.0| 0.03636363636363636|
|                 0.0|0.045454545454545456|
|                 0.0| 0.05454545454545454|
|                 0.0| 0.06363636363636363|
|                 0.0| 0.07272727272727272|
|                 0.0| 0.08181818181818182|
|                 0.0| 0.09090909090909091|
|                 0.0|                 0.1|
|0.001897533206831...|                 0.1|
|0.001897533206831...| 0.10909090909090909|
|0.001897533206831...| 0.11818181818181818|
|0.001897533206831...| 0.12727272727272726|
|0.001897533206831...| 0.13636363636363635|
|0.001897533206831...| 0.14545454545454545|
|0.001897533206831...| 0.15454545454545454|
|0.001897533206831...| 0.1636363

In [96]:
newdata = spark.read.csv('data_new.csv',header=True,inferSchema=True, sep=',')
new_data = featureassembler.transform(newdata)
pred_results=regressor.evaluate(new_data)
pred_results.predictions.take(5)

AnalysisException: cannot resolve 'Churn' given input columns: [Account_Manager, Age, Company, Independent Features, Location, Names, Num_Sites, Onboard_date, Total_Purchase, Years, prediction, probability, rawPrediction];
'Project [prediction#3012, unresolvedalias(cast('Churn as double), None), 1.0 AS 1.0#3042]
+- Project [Names#2945, Age#2946, Total_Purchase#2947, Account_Manager#2948, Years#2949, Num_Sites#2950, Onboard_date#2951, Location#2952, Company#2953, Independent Features#2965, rawPrediction#2978, probability#2993, UDF(rawPrediction#2978) AS prediction#3012]
   +- Project [Names#2945, Age#2946, Total_Purchase#2947, Account_Manager#2948, Years#2949, Num_Sites#2950, Onboard_date#2951, Location#2952, Company#2953, Independent Features#2965, rawPrediction#2978, UDF(rawPrediction#2978) AS probability#2993]
      +- Project [Names#2945, Age#2946, Total_Purchase#2947, Account_Manager#2948, Years#2949, Num_Sites#2950, Onboard_date#2951, Location#2952, Company#2953, Independent Featu